# Problem: Write a custom Dataset and Dataloader to load from a CSV file

### Problem Statement
You are tasked with creating a **custom Dataset** and **Dataloader** in PyTorch to load data from a given `data.csv` file. The loaded data will be used to run a pre-implemented linear regression model.

### Requirements
1. **Dataset Class**:
   - Implement a class `CustomDataset` that:
     - Reads data from a provided `data.csv` file.
     - Stores the features (X) and target values (Y) separately.
     - Implements PyTorch's `__len__` and `__getitem__` methods for indexing.

2. **Dataloader**:
   - Use PyTorch's `DataLoader` to create an iterable for batch loading the dataset.
   - Support user-defined batch sizes and shuffling of the data.

In [1]:
import torch
import pandas as pd

torch.manual_seed(42)
X = torch.rand(100, 1) * 10  # 100 data points between 0 and 10
y = 2 * X + 3 + torch.randn(100, 1)  # Linear relationship with noise

# Save the generated data to data.csv
data = torch.cat((X, y), dim=1)
df = pd.DataFrame(data.numpy(), columns=['X', 'y'])
df.to_csv('data.csv', index=False)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# TODO: Add the missing code
class LinearRegressionDataset(Dataset):
    def __init__(self, csv_file_path: str):
        self.data = pd.read_csv(csv_file_path)

        self.X = torch.tensor(self.data["X"].to_list(), dtype=torch.float32).view(-1, 1)
        self.y = torch.tensor(self.data["y"].to_list(), dtype=torch.float32).view(-1, 1)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # X, y = self.data.iloc[index].to_list()
        # return torch.tensor([X], dtype=torch.float32), torch.tensor([y], dtype=torch.float32)
        return self.X[index], self.y[index]
        

# Example usage of the DataLoader
dataset = LinearRegressionDataset('data.csv')
# TODO: Add the missing code
dataloader = DataLoader(
    dataset=dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0
)


In [15]:
dataset[1]

(tensor([9.1500]), tensor([21.8833]))

In [16]:
# Define the Linear Regression Model
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # Single input and single output

    def forward(self, x):
        return self.linear(x)

# Initialize the model, loss function, and optimizer
model = LinearRegressionModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    for batch_X, batch_y in dataloader:
        # Forward pass
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Log progress every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [100/1000], Loss: 0.6023
Epoch [200/1000], Loss: 0.3635
Epoch [300/1000], Loss: 0.3494
Epoch [400/1000], Loss: 1.2853
Epoch [500/1000], Loss: 0.2914
Epoch [600/1000], Loss: 0.8652
Epoch [700/1000], Loss: 0.3524
Epoch [800/1000], Loss: 0.6754
Epoch [900/1000], Loss: 0.3911
Epoch [1000/1000], Loss: 0.4965


In [17]:
# Display the learned parameters
[w, b] = model.linear.parameters()
print(f"Learned weight: {w.item():.4f}, Learned bias: {b.item():.4f}")

# Testing on new data
X_test = torch.tensor([[4.0], [7.0]])
with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for {X_test.tolist()}: {predictions.tolist()}")

Learned weight: 1.9692, Learned bias: 3.2260
Predictions for [[4.0], [7.0]]: [[11.102598190307617], [17.01007652282715]]
